# Clustering v0.1
This notebook outlines how the features derived in Feature Extraction v0.1 are used to find clusters of passenger travel patterns across bus services. The key idea of the clustering is that the clusters will capture services where passengers have similar travel patterns and one could apply a ""global" intervention on the entire cluster to improve bus services. 

As mentioned in Feature Extraction v0.1, the features are of uneven length. Some are as short as 12 dimensions while others can be as long as 65. As such, traditional methods of distance measures between vectors fail as they assume feature vectors of even length. However, a feature vector of 60 dimensions can have a similar feature waveform with one that is of 20 dimensions. Ideally, they should fall in the same cluster.

Here, I use **Dynamic Time Warping (DTW)** as the form of distance measure.  DTW is a time-series analysis concept which measures similarity between two temporal sequences, which may vary in speed. For instance, similarities in walking could be detected using DTW, even if one person was walking faster than the other, or if there were accelerations and decelerations during the course of an observation. DTW has been applied to temporal sequences of video, audio, and graphics data.In short, any data that can be turned into a linear sequence can be analyzed with DTW. 

Since the feature vectors created are linear sequences of bus stops, DTW is quite apt for measuring similarity between feature vectors of different lengths. Using DTW, the distance matrix can be generated which in turn can be passed to traditional clustering approaches like K-Means, DBScan etc to find clusters. In this notebook, I use *Hierarchical Clustering* to find clusters. I find the number of clusters by determining where the biggest jump in distance occurs between aggregation of clusters.

In [ ]:
%run ./_env.py

In [ ]:
%run ./utils.py

In [ ]:
%run ./service_profile.py

In [ ]:
import numpy as np
import pandas as pd
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

The commands above load the external functions, classes and libraries needed by this notebook to run. The functions below connect to the Amazon S3 storage which contains the data and establishes a verified connection. After connecting, it mounts the volume onto the Databricks file system for access.

In [ ]:
# s3 bucket to use
bucket = S3Bucket("bus-v2-data", AWS_ACCESS_KEY, AWS_SECRET_KEY)
bucket.allowSpark().mount('s3', ignore_exception=True)

Define the data schemas for data source that contains the feature vectors and load the data with the schema defined.

In [ ]:
# resources
path = bucket.s3("workspace/amit/bus_dist_series_data_0.1.csv")
schema = StructType([
    StructField('Feature', DoubleType()),
    StructField('bus stop code', StringType()),
    StructField('direction', IntegerType()),
    StructField('service', StringType()),
])
df = spark.read.csv(path, header="true", schema=schema)
display(df)

Obtain the list of bus services and directions from the data. Convert the Spark dataframe to a Pandas dataframe for further analysis.

In [ ]:
# Obtain trunk bus services in data and corresponding directions
bus_srvc_direc = df.select('service','direction').distinct().rdd.map(lambda r: (r[0], r[1])).collect()

# Convert spark dataframe to pandas
dist_series_data = df.toPandas()

## Distance matrix computation
The distance matrix is initialized below with the columns and rows representing the various bus services and direction.

In [ ]:
# Compute DTW distances between all features and compute a 2D distance matrix for clustering
# Create empty pandas dataframe to store all the distances with the indices labelled by the bus service and direction
mat_sz = len(bus_srvc_direc)
I = pd.Index(bus_srvc_direc, name="")
C = pd.Index(bus_srvc_direc, name="")
dtw_distance_mat = pd.DataFrame(pd.np.zeros((mat_sz, mat_sz), dtype=np.float), index=I, columns=C)

The features are extracted for each bus service and direction from the Pandas dataframe into a list of numpy arrays. This is for easy indexing for distance computation later on.

In [ ]:
# Create feature list for easy indexing
feature_list = []
for item in bus_srvc_direc:
    feature = dist_series_data.loc[(dist_series_data['service'] == item[0]) & (dist_series_data['direction'] == item[1]), 'Feature'].values
    feature = feature.reshape((feature.shape[0], ))
    feature_list.append(feature)

The distance matrix is computed using a nested for loop over all the feature vectors. The distance returned is added to the matrix initialized earlier. To note, due to diagonal symmetry of the distance matrix, the code below only computes the upper half and replicates the results for the lower half.

In [ ]:
# Compute distance matrix for feature list
for i in range(0, len(feature_list)):
    for j in range(i + 1, len(feature_list)):
        distance, path = fastdtw(feature_list[i], feature_list[j], dist=euclidean)
        dtw_distance_mat.loc[(bus_srvc_direc[i][0], bus_srvc_direc[i][1]),(bus_srvc_direc[j][0],bus_srvc_direc[j][1])] = distance
        dtw_distance_mat.loc[(bus_srvc_direc[j][0], bus_srvc_direc[j][1]),(bus_srvc_direc[i][0],bus_srvc_direc[i][1])] = distance

The results are then saved to the S3 storage for clustering analysis without the need to repeatedly find the distance matrix.

In [ ]:
# Convert Pandas DataFrame to Spark DataFrame
df = spark.createDataFrame(dtw_distance_mat)

# Save Spark DataFrame to S3
df.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('s3a://bus-v2-data/workspace/amit/dtw_distance_mat_0.1.csv',mode="overwrite")

## Finding the clusters of passenger travel patterns across all bus services
The DTW distance matrix computed earlier is loaded from the S3 storage.

In [ ]:
# s3 bucket to use
bucket = S3Bucket("bus-v2-data", AWS_ACCESS_KEY, AWS_SECRET_KEY)
bucket.allowSpark().mount('s3', ignore_exception=True)

# util func
ws = lambda path: "/workspace/amit/" + path  # return path to my workspace

path = bucket.s3("workspace/amit/dtw_distance_mat_0.1.csv")
df = sqlContext.read.format('csv').options(header='true', inferSchema='true').load(path)

Load the libraries needed for the analysis.

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import squareform
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

Load the EZ-Link data and the route information data and initialize their data schemas. Also, the variables needed for filtering the data is initialized here. The reason we are doing so is that after clustering, we will need all this information to get the trips information for generation of the vega diagrams for the services belonging to each cluster.

In [ ]:
# resources
EZLINK = bucket.s3("data/ezlink-201702.parquet")
ROUTE = bucket.local("data/lta_scheduled_bus_routes_for_feb2017.csv")

from datetime import datetime, time

route_schema = dict(
    service="service",
    direction="direction",
    stop_code="BusStopCode",
    seq="BusStopSequence",
    km="km",
    dt_from="dt_from",
    dt_to="dt_to",
    time_format='%d/%m/%Y')

ezlink_schema = dict(
    src="BOARDING_STOP_STN",
    dst="ALIGHTING_STOP_STN",
    year="Year",
    bus_id="BUS_REG_NUM",
    trip_id="Bus_Trip_Num",
    journey_id="JOURNEY_ID",
    travel_mode="TRAVEL_MODE",
    service="Srvc_Number",
    direction="Direction",
    km="Ride_Distance",
    tap_in_time="tap_in_time",
    tap_out_time="tap_out_time")

route_valid_for_date = datetime(2017, 2, 14)
days_of_interest = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
am_peak = dict(start_time=time(7, 30), end_time=time(9, 30))

# route data
route = (Route.from_csv(ROUTE, **route_schema).valid_for(route_valid_for_date))

# ezlink data
ezlink_data = spark.read.parquet(EZLINK)

# Subset bus data
ezlink_data.createOrReplaceTempView('data_table')
ezlink_bus_data = sqlContext.sql('select * from data_table where TRAVEL_MODE="Bus"')
ezlink = Ezlink(ezlink_bus_data, **ezlink_schema)

In [ ]:
# Convert spark dataframe to pandas
dtw_distance_mat = df.toPandas()
ezlink_bus_srvc = tuple(dtw_distance_mat.columns.values)
idx = dict(enumerate(ezlink_bus_srvc, start=0))
dtw_distance_mat.rename(index=idx, inplace=True)

Convert the distance matrix to a form required by the sci-kit learn package.

In [ ]:
dtw_distance_mat_condensed = squareform(dtw_distance_mat)

Perform hierarchical clustering on the distance matrix and generate the dendogram to identify what the suitable number of clusters should be.

In [ ]:
# generate the linkage matrix
Z = linkage(dtw_distance_mat_condensed, 'ward')
plt.figure(figsize=(100, 40))
plt.title('Hierarchical Clustering Ward Linkage Dendrogram of Commuter-Bus Travel Patterns',fontsize=36)
plt.xlabel('sample index', fontsize=36)
plt.ylabel('distance', fontsize=36)
dn = dendrogram(Z, labels=dtw_distance_mat.index, leaf_font_size=16.)
display(plt.show())

With the cluster number determined, generate the feature plots and vega diagram plots of each bus service and direction within each cluster.

In [ ]:
plt.ioff()
fig = plt.figure(figsize=(100, 40))
for clust_num in range(5, 17):
    nodes = fcluster(Z, clust_num, criterion="maxclust")
    df = pd.DataFrame({
        "bus service direc": bus_srvc_direc,
        "feature": feature_list,
        "cluster": nodes
    })
    for cluster in range(1, clust_num + 1):
        df_sub = df.loc[df["cluster"] == cluster]
        dbutils.fs.mkdirs(bucket.s3("workspace/amit/plots_0.1/Clusters=" + str(clust_num) + "/" + str(cluster)))
        SAVED_PLOTS_LOCATION = bucket.local("workspace/amit/plots_0.1/Clusters=" + str(clust_num) + "/" + str(cluster))
        for ind, feat in enumerate(df_sub["feature"]):
            # Save feature plot of each service and direction for the cluster
            bus_stops = dist_series_data.loc[(dist_series_data['service'] == df_sub.iloc[ind, 0][0]) & (dist_series_data['direction'] == df_sub.iloc[ind, 0][1]), 'bus stop code'].values
            y_pos = np.arange(len(bus_stops))
            plt.bar(y_pos, feat, align='center', alpha=0.5)
            plt.xticks(y_pos, bus_stops, rotation='vertical')
            plt.title('Passenger Flow vs Stops for Service {} Direction {} in Cluster: {}'.format(df_sub.iloc[ind, 0][0], df_sub.iloc[ind, 0][1],cluster),fontsize=96)
            plt.xlabel('Bus Stops', fontsize=96)
            plt.ylabel('Normalized Passenger Flow', fontsize=96)
            ax = plt.gca()
            ax.tick_params(axis='both', which='major', labelsize=64)
            fig.savefig(SAVED_PLOTS_LOCATION + '/' + str(df_sub.iloc[ind, 0][0]) + '_' + str(df_sub.iloc[ind, 0][1]) + '.png', bbox_inches='tight')
            plt.clf()

            # Save corresponding vega diagram along with feature plot
            srvc = dict(service=df_sub.iloc[ind, 0][0],direction=df_sub.iloc[ind, 0][1])
            service_route = route.for_service(**srvc)
            trips = (ezlink.for_service(**srvc).in_days_of_week(days_of_interest).within_time_range(**am_peak).get_trips(service_route))
            edges = trips.to_edges(source="src_seq", target="dst_seq", value="pax")
            nodes = service_route.to_nodes(name="stop_code", index="seq", order="km")
            arc_html = vega(arc_diagram(edges=edges, nodes=nodes), render=False)
            arc_file = open(SAVED_PLOTS_LOCATION + '/' + str(df_sub.iloc[ind, 0][0]) + '_' + str(df_sub.iloc[ind, 0][1]) + '_vega_arc.html', "w")
            arc_file.write(arc_html)
            arc_file.close()

## Issues with the DTW approach for clustering
The problem with this approach was that the clusters generated were not understandable. On deeper analysis, I found that DTW basically normalizes phase shifts between feature vectors. So a feature vector with 1 triangular peak at around 25% of the journey will become highly similar with one that has 1 triangular peak at around 75% or 50% or even 90% because DTW compresses/expands the waveform during the distance computation to match. But, clearly, the feature vectors mentioned above are very different from one another.

There are no other distance measure approaches in literature at the moment that handle distance computation between features of uneven length.